Import Pakages

In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import openpyxl

Getting Tables From SQL Server

In [ ]:
server = 'den1.mssql7.gear.host' 
database = 'agridatabase' 
username = 'agridatabase' 
password = 'you_can_enter_now'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

df_asia = pd.read_sql("SELECT * FROM agridatabase..Production_Crops_E_Asia", conn)

Getting Tables From Local Disk

In [ ]:
# df_africa = pd.read_csv('production_crops_e_africa.csv')
df_americas = pd.read_csv('production_crops_e_americas.csv')
# df_asia = pd.read_csv('production_crops_e_asia.csv')
# df_europe = pd.read_csv('production_crops_e_europe.csv')
# df_oceania = pd.read_csv('production_crops_e_oceania.csv')

Droping null columns

In [ ]:
column = [col for col in df_americas.columns if 'f' not in col]

# df_africa = df_africa[column]
df_americas = df_americas[column]
# df_asia = df_asia[column]
# df_europe = df_europe[column]
# df_oceania = df_oceania[column]

In [ ]:
# df_asia[(df_asia.Area == 'Bangladesh') & (df_asia.Item == 'Tea') & (df_asia.Element == 'Production')].iloc[0, 2018-1954]
# df[df.area == country][df.item == item][df.element == element].iloc[0, year-1954]
# df_americas[(df_americas.area == 'Argentina') & (df_americas.item == 'Barley') & (df_americas.element == 'Production')]

Funtions For Transformation

In [ ]:
def get_value(df, country, item, year, element):
    try:
        value = df[(df.area == country) & (df.item == item) & (df.element == element)].iloc[0, year-1954]
    except:
        value = 0
    return 0 if pd.isnull(value) else int(value)

def transform_table(df, continent, elements):
    countries = df.area.unique()
    items = df.item.unique()
    trans_df = pd.DataFrame()
    raw_df = pd.DataFrame()
    for country in countries:
        print('running... ', country)
        for item in items:
            for year in np.arange(1961, 2020):
                Yield = get_value(df, country, item, year, elements[0])
                harvested = get_value(df, country, item, year, elements[1])
                production = get_value(df, country, item, year, elements[2])
                row = pd.DataFrame({'country':[country],'continent':[continent] , 'item':[item], 'year':[year], 'yield':[Yield], 'harvested':[harvested], 'production':[production]})
                raw_df = pd.concat([raw_df, row], ignore_index=True, axis=0)
            
            if not (raw_df[['yield', 'harvested', 'production']] == 0).all().all():
                trans_df = pd.concat([trans_df, raw_df], ignore_index=False, axis=0)
            raw_df = pd.DataFrame()
    return trans_df
        

Transforming

In [ ]:
elements = np.array(['Yield', 'Area harvested', 'Production'])

# trans_df_africa = transform_table(df_africa, 'Africa', elements)
trans_df_americas = transform_table(df_americas, 'America', elements)
# trans_df_asia = transform_table(df_asia, 'Asia', elements = [])
# trans_df_europe = transform_table(df_europe, 'Europe', elements)
# trans_df_oceania = transform_table(df_oceania, 'Oceania', elements)

Saving New Generated Databases

In [ ]:
# trans_df_africa.to_csv('trans_df_africa.csv', index=False)
# trans_df_africa.to_excel('trans_df_africa.xlsx', index=False)

trans_df_americas.to_csv('trans_df_americas.csv', index=False)
trans_df_americas.to_excel('trans_df_americas.xlsx', index=False)

# trans_df_asia.to_csv('trans_df_asia.csv', index=False)
# trans_df_asia.to_excel('trans_df_asia.xlsx', index=False)

# trans_df_europe.to_csv('trans_df_europe.csv', index=False)
# trans_df_europe.to_excel('trans_df_europe.xlsx', index=False)

# trans_df_oceania.to_csv('trans_df_oceania.csv', index=False)
# trans_df_oceania.to_excel('trans_df_oceania.xlsx', index=False)